# CS240: Computer Organization

## Blank, Fall 2017

### Assignment #6: Assembly Language Infrastructure, The Call Stack

This assignment is designed to:

* focus on the call stack
* implement call stack for general use in the LC-3
* understand limitations and utility of the call stack

**Assignment**

Your mission is to:

* design a call stack that will allow:
    * saving the (all/most) of the registers
    * calling a subroutine
    * optionally having a return value(s)
    * restoring the registers
* demonstrate that your system works by:
    * calling a recursive subroutine
    * calling a subroutine, which calls a subroutine
    * you can use the randomness subroutine, or other subroutines from the book

**Due**: one week from lab.

**Grading criteria**:

* Correctness of answer: 50%
* Explanation and Reflection: 50%

*Enter the names of the project members and project title in the next cell:*

Buzzfeed Quiz: Chloe Sheen, Seher Sethi, Taylor Cross

Enter your assembly code in the next cells. Add as many cells as you would like to explain how your code works and to explain it. 

In [39]:
;Cell 1: Basic Function

.ORIG x3000

    LEA R1, STACK ;; make sure that TOP as at top of STACK
    ST R1, TOP
    ;; Set some values 0-7 in registers as tests:
    AND R0, R0, #0
    ADD R0, R0, #1
    AND R1, R1, #0
    ADD R1, R1, #2
    AND R2, R2, #0
    ADD R2, R2, #3
    AND R3, R3, #0
    ADD R3, R3, #4
    AND R4, R4, #0
    ADD R4, R4, #5
    AND R5, R5, #0
    ADD R5, R5, #6
    AND R6, R6, #0
    ADD R6, R6, #7
    AND R7, R7, #0
    ADD R7, R7, #8
    ST R7, REG7 ;; Always do this if you want to save R7
    ;; FUNCTION CALL!
    ;; 1) Save Registers:
    JSR    PUSH
    ;; 2) Call function; setup ARGS in REGS if necessary:
    ;; --------------------------------------------------------------
    JSR    MAIN
    ;; --------------------------------------------------------------
    ;; 3) Look at registers now for results
          ;; any results?
    ;; 4) Now Restore Registers:
    JSR    POP
    LD     R7, REG7 ;; do this last!
    ;; 5) Continue ....
    HALT ;; R7 will get destroyed here!
    
MAIN: ST     R7, REG7 ;; always save R7 first!
      ;; Do some computing!
      ;; Destroy some registers
    AND R0, R0, #0
    ADD R0, R0, #1  ; R0 = 1
    AND R1, R1, #0
    ADD R1, R0, #1 ; R0 + R1 + 1 so R1 = 2
    AND R2, R2, #0
    ADD R2, R1, #1 ; R1 + R2 + 1 so R2 = 3
    AND R3, R3, #0
    ADD R3, R2, #1   ;R2 + R3 + 1 so R3 = 4
    AND R4, R4, #0
    ADD R4, R3, #1     ;R3 + R4 + 1 so R4=5

    ;; FUNCTION CALL!
    ;; 1) Save Registers:
         JSR    PUSH ;; destroys r7
    ;; 2) Call function; setup ARGS in REGS if necessary:
         AND     R4, R4, #0
         ADD     R4, R4, #3
         AND     R0, R0, #0
         ;; --------------------------------------------------------
         JSR    FUNCTION     ;; Begin! destroys r7
         ;; --------------------------------------------------------
    ;; 3) Look at registers now for results
          ;; The result is in R3
          ;; perhaps store in memory!
          ST R3, FUNCTION_ANSWER
    ;; 4) Now Restore Registers:
         JSR    POP ;; destroys r7
    ;; 5) Continue ....
         ;; overwrite reg if you wish!
         LD     R7, REG7 ;; do this last!
         RET

;; The Stack for the whole system:
TOP:    .FILL    STACK
STACK:  .BLKW    #100

FUNCTION:  LD R1, TWO
            AND R2, R2, #0 ;Clear R2
            ADD R2, R4, #0 ;Put value of R4 into R2
            AND R3, R3, #0

            AGAIN ADD R3, R3, R2
                  ADD R1, R1, #-1
                  BRp AGAIN
                  HALT
            RET
                                                                                                          
        TWO .FILL x0002
FUNCTION_ANSWER: .FILL #0

;;; ----------------------------------------- Stack
REG5: .FILL    #0
REG6: .FILL    #0
REG7: .FILL    #0
PUSH:  ST R6, REG6     ;; save R6 to variable REG6
       ST R5, REG5     ;; save R5 to variable REG5
                       ;; REG7 is in MEMORY already
                       ;; R6 and R5 are destroyed
       LD R6, TOP      ;; R6 is now TOP of stack
       STR R0, R6, #0      ;; push R0 on stack
       ADD R6, R6, #1  ;; incrementing the TOP of stack (R6)
       STR R1, R6, #0      ;; push R1 on stack
       ADD R6, R6, #1  ;; incr
       STR R2, R6, #0      ;; push R2 on stack
       ADD R6, R6, #1  ;; incr
       STR R3, R6, #0      ;; push R3 on stack
       ADD R6, R6, #1  ;; incr
       STR R4, R6, #0      ;; push R4 on stack
       ADD R6, R6, #1  ;; incr
       STR R5, R6, #0      ;; push R5 on stack
       ADD R6, R6, #1  ;; incr
       LD R5, REG6     ;; get R6 from MEMORY (destroys R5) → holds actual value of R6
       STR R5, R6, #0      ;; push R6 on stack
       ADD R6, R6, #1  ;; incr
       LD R5, REG7     ;; get R7 from MEMORY
       STR R5, R6, #0  ;; push R7 on stack 
       ADD R6, R6, #1  ;; incr
       ST R6, TOP      ;; save incremented TOP
       LD R6, REG6
       LD R5, REG5     ;; 
       RET

POP:   LD R6, TOP     ;; R6 is TOP of stack
       ADD R6, R6, #-1     ;; decr
       LDR  R1, R6, #0      ;; load R7
       ST  R1, REG7    ;; save to MEMORY
       ADD R6, R6, #-1     ;; decr
       LDR  R1, R6, #0      ;; load R6
       ST  R1, REG6    ;; save to MEMORY
       ADD R6, R6, #-1     ;; decr
       LDR  R5, R6, #0      ;; load R5
       ADD R6, R6, #-1     ;; decr
       LDR  R4, R6, #0      ;; load R4
       ADD R6, R6, #-1     ;; decr
       LDR  R3, R6, #0      ;; load R3
       ADD R6, R6, #-1     ;; decr
       LDR  R2, R6, #0      ;; load R2
       ADD R6, R6, #-1     ;; decr
       LDR  R1, R6, #0      ;; load R1
       ADD R6, R6, #-1     ;; decr
       LDR  R0, R6, #0      ;; load R0
       ST R6, TOP      ;; save decremented TOP
       LD R6, REG6, #0     ;; put R6 back the way it was
       RET
.END       



Assembled! Use %dis or %dump to examine; use %exe to run.


In [40]:
%exe

Computation completed
Instructions: 98
Cycles: 730 (0.000365 milliseconds)

Registers:
PC: x048E
N: 0 Z: 1 P: 0 
R0: x0000 R1: x0000 R2: x0003 R3: x0006 
R4: x0003 R5: x0006 R6: x0007 R7: x3099 


This shows our original function, which moves the value in R4 to R2, multiplies the value stored in R2 by 2 and store it in R3. We then save the registers, call the function with the example of storing the number 3 into R4.

In [41]:
;Cell 2 : Recursive Function

.ORIG x3000

    LEA R1, STACK ;; make sure that TOP as at top of STACK
    ST R1, TOP
    ;; Set some values 0-7 in registers as tests:
    AND R0, R0, #0
    ADD R0, R0, #1
    AND R1, R1, #0
    ADD R1, R1, #2
    AND R2, R2, #0
    ADD R2, R2, #3
    AND R3, R3, #0
    ADD R3, R3, #4
    AND R4, R4, #0
    ADD R4, R4, #5
    AND R5, R5, #0
    ADD R5, R5, #6
    AND R6, R6, #0
    ADD R6, R6, #7
    AND R7, R7, #0
    ADD R7, R7, #8
    ST     R7, REG7 ;; Always do this if you want to save R7
    ;; FUNCTION CALL!
    ;; 1) Save Registers:
    JSR    PUSH
    ;; 2) Call function; setup ARGS in REGS if necessary:
    ;; --------------------------------------------------------------
    JSR    MAIN
    ;; --------------------------------------------------------------
    ;; 3) Look at registers now for results
          ;; any results?
    ;; 4) Now Restore Registers:
    JSR    POP
    LD     R7, REG7 ;; do this last!
    ;; 5) Continue ....
    HALT ;; R7 will get destroyed here!
    

MAIN: ST     R7, REG7 ;; always save R7 first!
      ;; Do some computing!
      ;; Destroy some registers
    AND R0, R0, #0
    ADD R0, R0, #1  ; R0 = 1
    AND R1, R1, #0
    ADD R1, R0, #1 ; R0 + R1 + 1 so R1 = 2
    AND R2, R2, #0
    ADD R2, R1, #1 ; R1 + R2 + 1 so R2 = 3
    AND R3, R3, #0
    ADD R3, R2, #1   ;R2 + R3 + 1 so R3 = 4
    AND R4, R4, #0
    ADD R4, R3, #1     ;R3 + R4 + 1 so R4=5

    ;; FUNCTION CALL!
    ;; 1) Save Registers:
         JSR    PUSH ;; destroys r7
    ;; 2) Call function; setup ARGS in REGS if necessary:
         AND     R4, R4, #0
         ADD     R4, R4, #3
         AND     R0, R0, #0
         AND     R5, R5, #0
         ADD     R5, R5, #2 ; This is the counter
         AND     R3, R3, #0 ;This is where the answer is
         ;; --------------------------------------------------------
         JSR    MULT_FOUR     ;; Begin! destroys r7
         ;; --------------------------------------------------------
    ;; 3) Look at registers now for results
          ;; The result is in R3
          ;; perhaps store in memory!
          ST R3, FUNCTION_ANSWER
    ;; 4) Now Restore Registers:
         JSR    POP ;; destroys r7
    ;; 5) Continue ....
         ;; overwrite reg if you wish!
         LD     R7, REG7 ;; do this last!
         RET

;; The Stack for the whole system:
TOP:    .FILL    STACK
STACK:  .BLKW    #100


  FUNCTION:  LD R1, TWO
     AND R2, R2, #0 ;Clear R2
     ADD R2, R4, #0 ;Put value of R4 into R2
     AND R3, R3, #0

AGAIN ADD R3, R3, R2
            ADD R1, R1, #-1
            BRp AGAIN
            HALT

RET
                                                                                                          
TWO .FILL x0002

FUNCTION_ANSWER: .FILL #0

; CALLING RECURSIVE FUNCTION

MULT_FOUR: LD R1, TWO_AGAIN ;Two goes into R1 
           AND R2, R2, #0
           ADD R2, R4, #0 ;3 goes into R2
           
           AGAIN: ADD R3, R3, R2 ; The first time, Add 0 + R2 (3)
                  ADD R1, R1, #-1 ;Decrement R1
                  BRp AGAIN
                  ADD R5, R5, #-1 ;Decrement the counter
                  BRp MULT_FOUR
                  HALT
RET
           
TWO_AGAIN: .FILL #2

FUNCTION_ANSWER: .FILL #0
;;; ----------------------------------------- Stack
REG5: .FILL    #0
REG6: .FILL    #0
REG7: .FILL    #0
PUSH:  ST R6, REG6     ;; save R6 to variable REG6
       ST R5, REG5     ;; save R5 to variable REG5
                                 ;; REG7 is in MEMORY already
                       ;; R6 and R5 are destroyed
       LD R6, TOP     ;; R6 is now TOP of stack
       STR R0, R6, #0      ;; push R0 on stack
       ADD R6, R6, #1  ;; incrementing the TOP of stack (R6)
       STR R1, R6, #0      ;; push R1 on stack
       ADD R6, R6, #1  ;; incr
       STR R2, R6, #0      ;; push R2 on stack
       ADD R6, R6, #1  ;; incr
       STR R3, R6, #0      ;; push R3 on stack
       ADD R6, R6, #1  ;; incr
       STR R4, R6, #0      ;; push R4 on stack
       ADD R6, R6, #1  ;; incr
       STR R5, R6, #0      ;; push R5 on stack
       ADD R6, R6, #1  ;; incr
       LD R5, REG6     ;; get R6 from MEMORY (destroys R5) → holds actual value of R6
       STR R5, R6, #0      ;; push R6 on stack
       ADD R6, R6, #1  ;; incr
       LD R5, REG7     ;; get R7 from MEMORY
       STR R5, R6, #0  ;; push R7 on stack 
       ADD R6, R6, #1  ;; incr
       ST R6, TOP      ;; save incremented TOP
       LD R6, REG6
       LD R5, REG5     ;; 
       RET


POP:   LD R6, TOP     ;; R6 is TOP of stack
       ADD R6, R6, #-1     ;; decr
       LDR  R1, R6, #0      ;; load R7
       ST  R1, REG7    ;; save to MEMORY
       ADD R6, R6, #-1     ;; decr
       LDR  R1, R6, #0      ;; load R6
       ST  R1, REG6    ;; save to MEMORY
       ADD R6, R6, #-1     ;; decr
       LDR  R5, R6, #0      ;; load R5
       ADD R6, R6, #-1     ;; decr
       LDR  R4, R6, #0      ;; load R4
       ADD R6, R6, #-1     ;; decr
       LDR  R3, R6, #0      ;; load R3
       ADD R6, R6, #-1     ;; decr
       LDR  R2, R6, #0      ;; load R2
       ADD R6, R6, #-1     ;; decr
       LDR  R1, R6, #0      ;; load R1
       ADD R6, R6, #-1     ;; decr
       LDR  R0, R6, #0      ;; load R0
       ST R6, TOP      ;; save decremented TOP
       LD R6, REG6, #0     ;; put R6 back the way it was
       RET
.END


Assembled! Use %dis or %dump to examine; use %exe to run.


In [42]:
%exe

Computation completed
Instructions: 113
Cycles: 824 (0.000412 milliseconds)

Registers:
PC: x048E
N: 0 Z: 1 P: 0 
R0: x0000 R1: x0000 R2: x0003 R3: x000C 
R4: x0003 R5: x0000 R6: x0007 R7: x30A8 


The new function is this cell block is MULT_4. The intention of this function is to test the call stack's ability to handle recursive functions. The new argument added here is a "Counter" that is set to the value of 2. The reason we did not create a counter that has the value of 4, and simply conduct multiply 4 (repeated addition 4 times) was to see if our function could handle two branches as well as repeating a block of code that stood alone. Once the counter equals 0 (R5), the function halts and the program returns to the address in R7.

In [43]:
;Cell 3 : Subroutine calling a Subroutine

.ORIG x3000

    LEA R1, STACK ;; make sure that TOP as at top of STACK
    ST R1, TOP
    ;; Set some values 0-7 in registers as tests:
    AND R0, R0, #0
    ADD R0, R0, #1
    AND R1, R1, #0
    ADD R1, R1, #2
    AND R2, R2, #0
    ADD R2, R2, #3
    AND R3, R3, #0
    ADD R3, R3, #4
    AND R4, R4, #0
    ADD R4, R4, #5
    AND R5, R5, #0
    ADD R5, R5, #6
    AND R6, R6, #0
    ADD R6, R6, #7
    AND R7, R7, #0
    ADD R7, R7, #8
    ST R7, REG7 ;; Always do this if you want to save R7
    ;; FUNCTION CALL!
    ;; 1) Save Registers:
    JSR    PUSH
    ;; 2) Call function; setup ARGS in REGS if necessary:
    ;; --------------------------------------------------------------
    JSR    MAIN
    ;; --------------------------------------------------------------
    ;; 3) Look at registers now for results
          ;; any results?
    ;; 4) Now Restore Registers:
    JSR    POP
    LD     R7, REG7 ;; do this last!
    ;; 5) Continue ....
    HALT ;; R7 will get destroyed here!
    

MAIN: ST R7, REG7 ;; always save R7 first!
      ;; Do some computing!
      ;; Destroy some registers

    AND R0, R0, #0
    ADD R0, R0, #1  ; R0 = 1
    AND R1, R1, #0
    ADD R1, R0, #1 ; R0 + R1 + 1 so R1 = 2
    AND R2, R2, #0
    ADD R2, R1, #1 ; R1 + R2 + 1 so R2 = 3
    AND R3, R3, #0
    ADD R3, R2, #1   ;R2 + R3 + 1 so R3 = 4
    AND R4, R4, #0
    ADD R4, R3, #1     ;R3 + R4 + 1 so R4=5
  ;; FUNCTION CALL!
    ;; 1) Save Registers:
         JSR    PUSH ;; destroys r7
    ;; 2) Call function; setup ARGS in REGS if necessary:
         AND     R4, R4, #0
         ADD     R4, R4, #3
         AND     R0, R0, #0
         ;; --------------------------------------------------------
         JSR    FUNCTION     ;; Begin! destroys r7
         ;; --------------------------------------------------------
    ;; 3) Look at registers now for results
          ;; The result is in R3
          ;; perhaps store in memory!
          ST R3, FUNCTION_ANSWER
    ;; 4) Now Restore Registers:
         JSR    POP ;; destroys r7
    ;; 5) Continue ....
         ;; overwrite reg if you wish!
         LD     R7, REG7 ;; do this last!
         RET

;; The Stack for the whole system:
TOP:    .FILL    STACK
STACK:  .BLKW    #100

FUNCTION:  LD R1, TWO
            AND R2, R2, #0 ;Clear R2
            ADD R2, R4, #0 ;Put value of R4 into R2
            AND R3, R3, #0
            JSR FUNCTION_THREE

            AGAIN ADD R3, R3, R2
                  ADD R1, R1, #-1
                  BRp AGAIN
                  HALT
            RET
                                                                                                          
        TWO .FILL x0002
 
; Subroutine calling subroutine 
; negates value in R4  → makes 6 into -6
FUNCTION_THREE: NOT R4, R4
              		ADD R4, R4 #1
          			 RET

FUNCTION_ANSWER: .FILL #0

;;; ----------------------------------------- Stack
REG5: .FILL    #0
REG6: .FILL    #0
REG7: .FILL    #0
PUSH:  ST R6, REG6     ;; save R6 to variable REG6
       ST R5, REG5     ;; save R5 to variable REG5
                       ;; REG7 is in MEMORY already
                       ;; R6 and R5 are destroyed
       LD R6, TOP      ;; R6 is now TOP of stack
       STR R0, R6, #0      ;; push R0 on stack
       ADD R6, R6, #1  ;; incrementing the TOP of stack (R6)
       STR R1, R6, #0      ;; push R1 on stack
       ADD R6, R6, #1  ;; incr
       STR R2, R6, #0      ;; push R2 on stack
       ADD R6, R6, #1  ;; incr
       STR R3, R6, #0      ;; push R3 on stack
       ADD R6, R6, #1  ;; incr
       STR R4, R6, #0      ;; push R4 on stack
       ADD R6, R6, #1  ;; incr
       STR R5, R6, #0      ;; push R5 on stack
       ADD R6, R6, #1  ;; incr
       LD R5, REG6     ;; get R6 from MEMORY (destroys R5) → holds actual value of R6
       STR R5, R6, #0      ;; push R6 on stack
       ADD R6, R6, #1  ;; incr
       LD R5, REG7     ;; get R7 from MEMORY
       STR R5, R6, #0  ;; push R7 on stack 
       ADD R6, R6, #1  ;; incr
       ST R6, TOP      ;; save incremented TOP
       LD R6, REG6
       LD R5, REG5     ;; 
       RET

POP:   LD R6, TOP     ;; R6 is TOP of stack
       ADD R6, R6, #-1     ;; decr
       LDR  R1, R6, #0      ;; load R7
       ST  R1, REG7    ;; save to MEMORY
       ADD R6, R6, #-1     ;; decr
       LDR  R1, R6, #0      ;; load R6
       ST  R1, REG6    ;; save to MEMORY
       ADD R6, R6, #-1     ;; decr
       LDR  R5, R6, #0      ;; load R5
       ADD R6, R6, #-1     ;; decr
       LDR  R4, R6, #0      ;; load R4
       ADD R6, R6, #-1     ;; decr
       LDR  R3, R6, #0      ;; load R3
       ADD R6, R6, #-1     ;; decr
       LDR  R2, R6, #0      ;; load R2
       ADD R6, R6, #-1     ;; decr
       LDR  R1, R6, #0      ;; load R1
       ADD R6, R6, #-1     ;; decr
       LDR  R0, R6, #0      ;; load R0
       ST R6, TOP      ;; save decremented TOP
       LD R6, REG6, #0     ;; put R6 back the way it was
       RET
.END



Assembled! Use %dis or %dump to examine; use %exe to run.


In [44]:
%exe

Computation completed
Instructions: 102
Cycles: 755 (0.000378 milliseconds)

Registers:
PC: x048E
N: 0 Z: 1 P: 0 
R0: x0000 R1: x0000 R2: x0003 R3: x0006 
R4: xFFFD R5: x0006 R6: x0007 R7: x309A 


The new function FUNCTION_THREE is added to demonstrate a subroutine calling a subroutine. Our original FUNCTION calls FUNCTION THREE after carrying out its original function of multiplying the value in R2 by 2 and storing it in R3. We can see from the result that the value stored in R4 has been negated, and thus the subroutine calling a subroutine works.

# Reflection

The reflection cell is the most import cell in this lab... in all labs. In your reflection, you should:

* Reflect! Think and comment on what you have learned this lab, and this week. 
* Connect the ideas onto your own life, and experiences.
* Comment on what you found challenging, and what you found intuitive.

Put your reflections in the cell below. You should write more than a paragraph. Remember:

* don't indent paragraphs
* use Markdown highlighting 
* use the spell checker

This lab was challenging to get started. All three of us felt that we had just been introduced to assembly language after about two months of dealing with machine language and so it was a sudden transitional dive into writing lines in assembly and learning about complex concepts while trying to familiarize ourselves with the syntax itself. However, we went line by line through the given examples in the textbook and in the provided notebook and eventually did get a better grasp of the concepts and the language. 

We needed more practice writing smaller functions and seeing things that work before jumping into writing more complex algorithms, so we decided not to confuse ourselves even more and overwhelm ourselves by trying to translate the fibonacci algorithm into assembly language. Instead, we stuck to simpler arithmetic algorithms such as moving values from a register to another, multiplying values, multiplying values recursively using a counter, and negating a value. Taking these smaller blocks of code, we could eventually build our stack to calculate a simple factorial or even a fibonacci sequence, but I think this is a good starting point for us. 

We also struggled to find a good way to collaborate together, because our schedules weren't too compatible and none of us are very good with git. We ended up sharing a Google Doc together and used the comment function to discuss the bugs and our explanations, which was not ideal as it often destroyed the formatting. Hopefully we'll be able to meet up as a group more often rather than try to resort to Google Docs. It would be amazing if Jupyter had a live sharing option.